In [9]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import time
from PIL import Image


VGG_MEAN = [103.930,116.779,123.68]

In [10]:
class VGGNet():
    def __init__(self,data_dict):
        self.data_dict = data_dict
        
    def get_conv_fliter(self,name):
        return (tf.constant(self.data_dict[name][0]))
    
    def get_fc_weight(self,name):
        return (tf.constant(self.data_dict[name][0]))
    
    def get_bias(self,name):
        return (tf.constant(self.data_dict[name][1]))
    
    def conv_layer(self,x,name):
        with tf.name_scope(name):
            conv_w = self.get_conv_fliter(name)
            conv_b = self.get_bias(name)
            #输入x的shape = [bath_size,width,height,channel],stride是在每个维度的步长
            h = tf.nn.conv2d(x,conv_w,strides=[1,1,1,1],padding='SAME')
            h_add_b = tf.nn.bias_add(h,conv_b)
            h_add_b_act = tf.nn.relu(h_add_b)
        return h_add_b_act
    
    def pooling_layer(self,x,name):
        with tf.name_scope(name):
            return (tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME",name=name))
    
    def fc_layer(self,x,name,activation):
        with tf.name_scope(name):
            fc_w = self.get_fc_weight(name)
            fc_b = self.get_bias(name)
            h = tf.matmul(x,fc_w)
            h_add_b = tf.bias_add(h,fc_b)
            if activation is None:
                return (h_add_b)
            else:
                return (activation(h_add_b))
    
    def flatten_layer(self,x,name):
        with tf.name_scope(name):
            x_shape = x.get_shape().as_list()
            x_reshape = tf.reshape(x_shape,[x_shape[0],-1])
            return x_reshape
    
    def build(self,x_rgb):
        start_time = time.time()
        print('building ...')
        r,g,b = tf.split(x_rgb,num_or_size_splits=[1,1,1],axis=3)
        x_rgb = tf.concat([b-VGG_MEAN[0],g-VGG_MEAN[1],r-VGG_MEAN[2]],axis=3)
        
        
        self.conv1_1 = self.conv_layer(x_rgb, 'conv1_1')
        self.conv1_2 = self.conv_layer(self.conv1_1, 'conv1_2')
        self.pool1 = self.pooling_layer(self.conv1_2, 'pool1')

        self.conv2_1 = self.conv_layer(self.pool1, 'conv2_1')
        self.conv2_2 = self.conv_layer(self.conv2_1, 'conv2_2')
        self.pool2 = self.pooling_layer(self.conv2_2, 'pool2')

        self.conv3_1 = self.conv_layer(self.pool2, 'conv3_1')
        self.conv3_2 = self.conv_layer(self.conv3_1, 'conv3_2')
        self.conv3_3 = self.conv_layer(self.conv3_2, 'conv3_3')
        self.pool3 = self.pooling_layer(self.conv3_3, 'pool3')

        self.conv4_1 = self.conv_layer(self.pool3, 'conv4_1')
        self.conv4_2 = self.conv_layer(self.conv4_1, 'conv4_2')
        self.conv4_3 = self.conv_layer(self.conv4_2, 'conv4_3')
        self.pool4 = self.pooling_layer(self.conv4_3, 'pool4')

        self.conv5_1 = self.conv_layer(self.pool4, 'conv5_1')
        self.conv5_2 = self.conv_layer(self.conv5_1, 'conv5_2')
        self.conv5_3 = self.conv_layer(self.conv5_2, 'conv5_3')
        self.pool5 = self.pooling_layer(self.conv5_3, 'pool5')
        
        print("the model is finished with %5d seconds" % (time.time()-start_time))
    

In [12]:
vgg16_npy_path = './vgg16.npy'
content_img_path = './he.jpg'
style_img_path = './start.jpg'
num_steps = 1000
learning_rate = 10
lambda_c = 0.2
lambda_s = 5000
output_path = './the_sencond_run'
if not os.path.exists(output_path):
    os.mkdir(output_path)

In [13]:
def initial_result(shape,mean,stddev):
    initial = tf.truncated_normal(shape,mean = mean,stddev = stddev)
    return tf.Variable(initial)

def read_img(image_path):
    image = Image.open(image_path)
    np_image = np.array(image)
    np_imge_reshape = np.asarray([np_image])
    return np_imge_reshape

def gram_matrix(x):
    b,w,h,c = x.get_shape().as_list()
    x_reshape = x.reshape(b,w*h,c)
    gram = tf.matmul(x_reshape,x_reshape,adjoint_a=True) / tf.constant(w*h*c,tf.float32)
    return gram
    


result = initial_result([1,224,224,3],127.5,20)

content_val = read_img(content_img_path)
style_val = read_img(style_img_path)

content = tf.placeholder(tf.float32,[1,224,224,3])
style = tf.placeholder(tf.float32,[1,224,224,3])

data_dict = np.load(vgg16_npy_path,allow_pickle=True,encoding='latin1').item()

vgg_for_content = VGGNet(data_dict)
vgg_for_style = VGGNet(data_dict)
vgg_for_result = VGG(data_dict)

vgg_for_content.build(content)
vgg_for_style.build(style)
vgg_for_result.build(result)

feature_of_content = [vgg_for_content.conv1_2]
feature_of_content_result = [vgg_for_result.conv1_2]

feature_of_style = [vgg_for_style.conv4_3]
gram_of_style =[gram_matrix(feature) for feature in feature_of_style]
feature_of_style_result = [vgg_for_result.conv4_3]
gram_of_style_result =[gram_matrix(feature) for feature in feature_of_style_result]

content_loss = tf.zeros(1,tf.float32)

for c,c_ in zip(feature_of_content,feature_of_content_result):
    content_loss += tf.reduce_mean((c-c_)**2 ,axis=[1,2,3])
    
style_loss = tf.zeros(1,tf.float32)

for s,s_ in zip(gram_of_style,gram_of_style_result):
    style_loss += tf,reduce_mean((s-s_)**2,axis=[1,2])

loss = lambda_c*content_loss + lambda_s *style_loss

train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

init_op = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(num_steps):
        loss_val,content_loss_val,style_loss_val,_ = \
                 sess.run([loss,content_loss,style_loss,train_op],
                 feed_dict = {content:content_val , style:style_val})
        print("step:%d,loss_val:%8.4f,content_loss:%8.4f,style_loss:%8.4f",%(step+1,
                                                                            loss_val[0],
                                                                            content_loss_val[0],
                                                                            style_loss_val[0]))
        result_img_path = os.path.joim(output_path,'result_img_%d.jpg'%(step+1))
        result_val = result.eval(sess)[0]
        result_val = np.clip(result_val,0,255)
        img_arr = np.asarray(result_val,np.uint8)
        img = Image.fromarray(img_arr)
        img_save(result_img_path)  

NameError: name 'mean' is not defined